In [ ]:
from __future__ import print_function, \
    division, \
    absolute_import

import os

import sep
import h5py

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)

from astropy.io import fits
from astropy.modeling import models, fitting

#---------------------------------------------------------------------------#
#User imports
import sys
sys.path.append('/Users/fardila/Documents/Github/kungpao')
from kungpao.galsbp import galSBP
from kungpao.display import display_single, random_cmap


In [2]:
# For Kungpao
x_images = '/Users/fardila/anaconda/envs/hsc_hydro/iraf/bin.macosx/x_images.e'
x_ttools = '/Users/fardila/anaconda/envs/hsc_hydro/iraf_extern/tables/bin.macosx/x_ttools.e'
x_isophote = '/Users/fardila/anaconda/envs/hsc_hydro/iraf_extern/stsdas/bin.macosx/x_isophote.e'
#---------------------------------------------------------------------------#

#---------------------------------------------------------------------------#
# About the Colormaps
IMG_CMAP = plt.get_cmap('viridis')
IMG_CMAP.set_bad(color='black')

SEG_CMAP = random_cmap(ncolors=512, background_color=u'white')
SEG_CMAP.set_bad(color='white')
SEG_CMAP.set_under(color='white')

In [3]:
from pyraf import iraf

iraf.tables()
iraf.stsdas()
iraf.analysis()
iraf.isophote()

iraf.unlearn('ellipse')
iraf.unlearn('bmodel')



      +------------------------------------------------------------+
      |             Space Telescope Tables Package                 |    
      |                  TABLES Version 3.17                       |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      +------------------------------------------------------------+
tables/:
 fitsio/        tbplot/         tobsolete/      ttools/


      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |    
      |                   STSDAS Version 3.17                      |
      |                   

In [ ]:
# Here, we use the stsdas.isophote.analysis.bmodel function to reconstruct 
# a 2-D model using the isophote information 
# So that we can subtract it from the original image, and see how well it does.

try: 
    os.remove(fits_prefix + '_cen_ellip_1.fits')
except Exception: 
    pass

iraf.bmodel(parent=fits_prefix + '_cen.fits', 
            table=iso_cen_1_bin,
            output=fits_prefix + '_cen_ellip_1.fits',
            minsma=0.0,
            highar='no')

img_cen_ellip_1 = fits.open(fits_prefix + '_cen_ellip_1.fits')[0].data

fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(hspace=0.0, wspace=0.0, 
                    left=0.0, bottom=0.0,
                    top=1.0, right=1.0)
ax1 = fig.add_subplot(1, 1, 1)

ax1 = display_single(img_cen - img_cen_ellip_1, 
                     ax=ax1,
                     contrast=0.15,
                     scale_bar_length=20.0,
                     scale_bar_loc='right',
                     stretch='linear',
                     pixel_scale=1.0,
                     physical_scale=3.0,
                     color_bar=True)

# Overplot a subsample of isophotes on the image
iso_cen_1_ellip = galSBP.convIso2Ell(iso_cen_1)
for ii, e in enumerate(iso_cen_1_ellip):
    if (ii % 3 == 0):
        ax1.add_artist(e)
        e.set_clip_box(ax1.bbox)
        e.set_alpha(0.6)
        e.set_edgecolor('r')
        e.set_facecolor('none')
        e.set_linewidth(2.0)

# Here, we can see the model becomes off-center quickly, and it reveals many 
# diffuse features underneath.